In [ ]:
!pip install transformers datasets accelerate bitsandbytes peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
!pip install python-docx pypdf
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.2/304.2 kB 21.2 MB/s eta 0:00:00


In [ ]:
import os
import re
import json
import gc
import pickle
import torch
from google.colab import files
from datasets import Dataset, concatenate_datasets
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    logging as hf_logging,
)
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document as LC_Doc
from docx import Document as DocxDocument

# Silence HF warnings
hf_logging.set_verbosity_error()

# 1) Mitigate GPU fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"

In [ ]:
# 2) Upload and parse instruction JSON
uploaded = files.upload()
instr_fname = next(iter(uploaded))
raw = uploaded[instr_fname].decode("utf-8")
# extract JSON array
start, end = raw.find("["), raw.rfind("]") + 1
js = raw[start:end]
js = re.sub(r",\s*\]$", "]", js)  # remove trailing comma
instr_pairs = json.loads(js)
instr_ds = Dataset.from_list([
    {"text": f"User: {ex['user_query']}\nAssistant: {ex['response']}\n"}
    for ex in instr_pairs
])


Saving Real-Estate Queries.txt to Real-Estate Queries (1).txt


In [ ]:
# 3) Upload and read Word document
uploaded = files.upload()
docx_fname = next(iter(uploaded))
doc = DocxDocument(docx_fname)
raw_text = "\n".join(p.text for p in doc.paragraphs)

Saving Real Estate Services.docx to Real Estate Services.docx


In [ ]:
def clean_healthcare_text(text: str) -> str:
    text = re.sub(r'\xa0(\d+)', r' \1', text)                     # Fix nbsp before numbers
    text = re.sub(r'(?<=\D)\$(\d)', r' \1', text)                 # Space before currency
    text = re.sub(r'\b(\d+)\s+([KM])\b', r'\1\2', text)           # 150 K → 150K
    text = re.sub(r'\$(\d+)\s+([KM])\b', r'\1\2', text)           # $10 K → $10K
    text = re.sub(r'(\d)(vs\.?)(\d)', r'\1 vs \3', text)          # 75(vs.150 → 75 vs 150
    text = re.sub(r'(\d)\s?–\s?(\d)', r'\1-\2', text)             # En-dash formatting
    text = re.sub(r'(\d+)([A-Za-z]{2,})', r'\1 \2', text)         # 50Kper → 50K per
    text = re.sub(r'([a-zA-Z])(\d)', r'\1 \2', text)              # letters next to digits
    text = re.sub(r'\b(e\.)\s?(g\.)\s?,', r'e.g.,', text)         # normalize e.g.,
    text = re.sub(r'\b(i\.)\s?(e\.)\s?,', r'i.e.,', text)         # normalize i.e.,
    text = re.sub(r'\s+', ' ', text)                              # collapse whitespace
    text = re.sub(r'(?<=\D)\.(?=\S)', '. ', text).strip()         # ensure space after sentences
    text = re.sub(r'[^A-Za-z0-9\s]+', ' ', text)                  # remove all punctuation/symbols
    text = re.sub(r'\s+', ' ', text).strip()                      # collapse spaces again
    return text

In [ ]:
cleaned = clean_healthcare_text(raw_text)


In [ ]:
cleaned

'Real Estate Services Overview Real estate services encompass property transactions leasing management and technology driven innovations This sector merges physical assets with digital tools like AI valuation virtual tours and IoT enabled smart buildings Key players include Zillow residential listings CBRE commercial real estate and Proptech startups like Opendoor Regulatory frameworks e g Fair Housing Act and sustainability mandates LEED certification shape operations 1 Property Transactions Residential Real Estate Buying Selling Homes Agents facilitate transactions using MLS Multiple Listing Service databases Platforms like Redfin offer 1 listing fees vs traditional 2 5 3 AI tools like Zillow s Zestimate provide instant home valuations 1 9 accuracy Auctions Sotheby s Concierge Auctions sell luxury properties digitally Auction com uses blockchain to verify bidder identities and streamline payments Financing Mortgage brokers connect buyers with lenders e g Rocket Mortgage s 8 minute pr

In [ ]:
# 5) Chunk into ~800-char slices
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=80)
chunks = splitter.split_text(cleaned)
content_ds = Dataset.from_list([{"text": c} for c in chunks])

In [ ]:
# 1) Load tokenizer & base model in FP16
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    "EleutherAI/gpt-neo-1.3B",
    device_map="auto",
    torch_dtype=torch.float16,   # 16-bit precision
)

In [ ]:
# 2) Prepare model for LoRA:
#    - disable caching for checkpointing compatibility
#    - enable gradient checkpointing
#    - freeze all base-model parameters
model.config.use_cache = False
model.gradient_checkpointing_enable()
for param in model.parameters():
    param.requires_grad = False

In [ ]:
# 3) Inject LoRA adapters (only these will train)
lora_cfg = LoraConfig(
    task_type="CAUSAL_LM",
    inference_mode=False,
    r=16,               # LoRA rank
    lora_alpha=32,
    lora_dropout=0.1,
)
model = get_peft_model(model, lora_cfg)

In [ ]:
# Verify that only LoRA params are trainable
trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total     = sum(p.numel() for p in model.parameters())
print(f"Trainable params: {trainable:,} / {total:,}")

Trainable params: 3,145,728 / 1,318,721,536


In [ ]:
def tokenize_and_label(ex):
    tok = tokenizer(
        ex["text"],
        padding="max_length",
        truncation=True,
        max_length=512,
    )
    # For causal LM, labels = copy of input_ids
    tok["labels"] = tok["input_ids"].copy()
    return tok

tok_instr   = instr_ds.map(tokenize_and_label, batched=True, remove_columns=["text"])
tok_content = content_ds.map(tokenize_and_label, batched=True, remove_columns=["text"])

Map:   0%|          | 0/1252 [00:00<?, ? examples/s]

Map:   0%|          | 0/13 [00:00<?, ? examples/s]

In [ ]:
train_dataset = concatenate_datasets([tok_instr, tok_content])

/usr/local/lib/python3.11/dist-packages/datasets/table.py:1395: FutureWarning: promote has been superseded by promote_options='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [ ]:
from transformers import DataCollatorForLanguageModeling
# 5) Use a data collator that handles causal LM labels
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)


In [ ]:
# 5) Training arguments (FP16 + no eval split)
training_args = TrainingArguments(
    output_dir="./gpt_neo_fp16_lora",
    num_train_epochs=6,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    fp16=True,
    optim="adamw_torch",
    learning_rate=2e-4,
    weight_decay=0.01,
    max_grad_norm=1.0,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    eval_strategy="no",
    save_steps=200,
    save_total_limit=3,
    logging_steps=50,
    report_to="none",
    load_best_model_at_end=False,
)

In [ ]:
# 6) Trainer setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)


In [ ]:
torch.cuda.empty_cache()
gc.collect()

432

In [ ]:
# 7) Train
trainer.train()
# 8) Save LoRA adapters & tokenizer
model.save_pretrained("./gpt_neo_fp16_lora")
tokenizer.save_pretrained("./gpt_neo_fp16_lora")
print("✅ FP16 + LoRA fine-tuning complete.")

{'loss': 4.5699, 'grad_norm': 29.9731502532959, 'learning_rate': 8.958333333333335e-05, 'epoch': 0.315955766192733}
{'loss': 0.3717, 'grad_norm': 0.06511569023132324, 'learning_rate': 0.00019375000000000002, 'epoch': 0.631911532385466}
{'loss': 0.2901, 'grad_norm': 0.059939321130514145, 'learning_rate': 0.00019852286972286428, 'epoch': 0.9478672985781991}
{'loss': 0.2534, 'grad_norm': 0.09758349508047104, 'learning_rate': 0.00019375879412266336, 'epoch': 1.259083728278041}
{'loss': 0.2325, 'grad_norm': 0.0921909436583519, 'learning_rate': 0.00018586097224101766, 'epoch': 1.575039494470774}
{'loss': 0.2297, 'grad_norm': 0.08089960366487503, 'learning_rate': 0.00017509337686239727, 'epoch': 1.890995260663507}
{'loss': 0.2111, 'grad_norm': 0.10988401621580124, 'learning_rate': 0.00016181589862206052, 'epoch': 2.2022116903633493}
{'loss': 0.2112, 'grad_norm': 0.11704915761947632, 'learning_rate': 0.00014647231720437686, 'epoch': 2.518167456556082}
{'loss': 0.2169, 'grad_norm': 0.1325779259

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel, PeftConfig

# 1) Paths
MODEL_DIR = "./gpt_neo_fp16_lora"

In [ ]:
# 2) Load PEFT config & base model
peft_config = PeftConfig.from_pretrained(MODEL_DIR)
base_model = AutoModelForCausalLM.from_pretrained(
    peft_config.base_model_name_or_path,
    device_map="auto",
    torch_dtype=torch.float16,
)

In [ ]:
# 3) Wrap with LoRA adapters
model = PeftModel.from_pretrained(base_model, MODEL_DIR)
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoForCausalLM(
      (transformer): GPTNeoModel(
        (wte): Embedding(50257, 2048)
        (wpe): Embedding(2048, 2048)
        (drop): Dropout(p=0.0, inplace=False)
        (h): ModuleList(
          (0-23): 24 x GPTNeoBlock(
            (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
            (attn): GPTNeoAttention(
              (attention): GPTNeoSelfAttention(
                (attn_dropout): Dropout(p=0.0, inplace=False)
                (resid_dropout): Dropout(p=0.0, inplace=False)
                (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
                (v_proj): lora.Linear(
                  (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                  (lora_dropout): ModuleDict(
                    (default): Dropout(p=0.1, inplace=False)
                  )
                  (lora_A): ModuleDict(
                    (default): Linear(in_f

In [ ]:
# 4) Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# 5) Generation function
def generate_response(query: str, max_new_tokens=100) -> str:
    prompt = f"User: {query}\nAssistant:"
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=512,
    ).to(model.device)
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            no_repeat_ngram_size=2,
            top_p=0.9,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id,
        )
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    return text.split("Assistant:")[-1].strip()


In [ ]:
# 6) Example usage
if __name__ == "__main__":
    queries = [
        "Hi! How you doing?",
    ]
    for q in queries:
        print(f"> {q}\n{generate_response(q)}\n")

    # Or enter an interactive loop:
    print("Enter 'exit' to quit.")
    while True:
        q = input("You: ").strip()
        if q.lower() in {"exit","quit"}:
            break
        print("Bot:", generate_response(q))

> Hi! How you doing?
Hey, you’re home.

Enter 'exit' to quit.
You: hi brother!
Bot: Hi! Who’s asking?


KeyboardInterrupt: Interrupted by user